# Building ETL Pipeline
1. Extract from CSV
2. Transform + clean data
3. Create db
4. Load data to db

# Extract data from csv

In [3]:
import pandas as pd
import numpy as np
import psycopg2 as sql
from sqlalchemy import create_engine

C:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [108]:
df = pd.read_csv("amazon_prime_titles.csv")
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [109]:
df.tail()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
9663,s9664,Movie,Pride Of The Bowery,Joseph H. Lewis,"Leo Gorcey, Bobby Jordan",NaN,NaN,1940,7+,60 min,Comedy,New York City street principles get an East Si...
9664,s9665,TV Show,Planet Patrol,NaN,"DICK VOSBURGH, RONNIE STEVENS, LIBBY MORRIS, M...",NaN,NaN,2018,13+,4 Seasons,TV Shows,"This is Earth, 2100AD - and these are the adve..."
9665,s9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaN,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou..."
9666,s9667,TV Show,Maradona: Blessed Dream,NaN,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",NaN,NaN,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ..."
9667,s9668,Movie,Harry Brown,Daniel Barber,"Michael Caine, Emily Mortimer, Joseph Gilgun, ...",NaN,NaN,2010,R,103 min,"Action, Drama, Suspense","Harry Brown, starring two-time Academy Award w..."


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7585 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


# Transform and clean data

## Delete unused columns

In [111]:
df.drop(columns=['description', 'cast', 'date_added'], inplace=True)

## Deal with duplicates

In [112]:
df.duplicated().sum()

np.int64(0)

## Deal with missing data

In [120]:
df.isna().sum()

show_id         0
type            0
title           0
director        0
country         0
release_year    0
rating          0
duration        0
listed_in       0
dtype: int64

Fill nan values for director with unknown

In [114]:
df.fillna({"director":"Unknown"}, inplace=True)

In [115]:
df = df.drop(df[df['director'].str.len().fillna(0) < 2].index)

Fill country according to the director

In [116]:
df.shape

(9652, 9)

Fills missing values in the 'country' column by using the first known country associated with each director.


In [117]:
df['country'] = df['country'].fillna(
    df.groupby('director')['country'].transform('first')
)

Fill rest of the countries with the most popular

In [118]:
df['country']=df['country'].fillna(df['country'].mode()[0])

Fill missing ratings with the most popular one

In [119]:
df['rating']=df['rating'].fillna(df['rating'].mode()[0])

## Delete index

In [121]:
df.drop(columns=['show_id'], inplace=True)


## Rename columns

In [122]:
df.rename(columns={'show_id':'id', "listed_in": "genre"}, inplace=True)

## Transform rating

In [123]:
df['rating'].unique()

array(['13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+', 'TV-PG',
       '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'G', 'PG',
       'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE'], dtype=object)

In [124]:
df["rating"]=df["rating"].replace({
    "All": "0+",
    "R": "17+",
    "TV-Y": "0+",
    'TV-Y7': "7+",
    'NR': 'UNRATED',
    'TV-PG': "10+",
    'TV-14': "14+",
    'TV-NR': 'UNRATED',
    'TV-G': "0+",
    'PG-13': "13+",
    'TV-MA': "17+",
    'G':"0+",
    'PG': "8+",
    'NC-17': "18+",
    '16': "16+",
    'AGES_16_': "16+",
    'AGES_18_': "18+",
    'ALL_AGES': "0+",
    'NOT_RATE': "UNRATED"
})

## Transform duration

In [125]:
df["duration"].unique()

array(['113 min', '110 min', '74 min', '69 min', '45 min', '52 min',
       '98 min', '131 min', '87 min', '92 min', '88 min', '93 min',
       '94 min', '46 min', '96 min', '1 Season', '104 min', '62 min',
       '50 min', '3 Seasons', '2 Seasons', '86 min', '36 min', '37 min',
       '103 min', '9 min', '18 min', '14 min', '20 min', '19 min',
       '22 min', '60 min', '6 min', '54 min', '5 min', '84 min',
       '126 min', '125 min', '109 min', '89 min', '85 min', '56 min',
       '40 min', '111 min', '33 min', '34 min', '95 min', '99 min',
       '78 min', '4 Seasons', '77 min', '55 min', '53 min', '115 min',
       '58 min', '49 min', '135 min', '91 min', '64 min', '59 min',
       '48 min', '122 min', '90 min', '102 min', '65 min', '114 min',
       '136 min', '70 min', '138 min', '100 min', '480 min', '4 min',
       '30 min', '152 min', '68 min', '57 min', '7 Seasons', '31 min',
       '151 min', '149 min', '9 Seasons', '141 min', '121 min', '79 min',
       '140 min', '51 min'

Take just number from duration in minutes

In [126]:
df["duration_in_min"] = [ dur.split("min")[0].strip() if "min" in dur else 0 for dur in df["duration"] ]

Take just number from duration in seasons

In [127]:
df["duration_in_seasons"] = [ dur.split("Season")[0].strip() if "Season" in dur else 0 for dur in df["duration"] ]

In [128]:
df.head()

,type,title,director,country,release_year,rating,duration,genre,duration_in_min,duration_in_seasons
0,Movie,The Grand Seduction,Don McKellar,Canada,2014,13+,113 min,"Comedy, Drama",113,0
1,Movie,Take Care Good Night,Girish Joshi,India,2018,13+,110 min,"Drama, International",110,0
2,Movie,Secrets of Deception,Josh Webber,United States,2017,13+,74 min,"Action, Drama, Suspense",74,0
3,Movie,Pink: Staying True,Sonia Anderson,United States,2014,13+,69 min,Documentary,69,0
4,Movie,Monster Maker,Giles Foster,United Kingdom,1989,13+,45 min,"Drama, Fantasy",45,0


In [129]:
df.tail()

,type,title,director,country,release_year,rating,duration,genre,duration_in_min,duration_in_seasons
9663,Movie,Pride Of The Bowery,Joseph H. Lewis,Italy,1940,7+,60 min,Comedy,60,0
9664,TV Show,Planet Patrol,Unknown,Italy,2018,13+,4 Seasons,TV Shows,0,4
9665,Movie,Outpost,Steve Barker,Italy,2008,17+,90 min,Action,90,0
9666,TV Show,Maradona: Blessed Dream,Unknown,Italy,2021,17+,1 Season,"Drama, Sports",0,1
9667,Movie,Harry Brown,Daniel Barber,Italy,2010,17+,103 min,"Action, Drama, Suspense",103,0


In [130]:
df.drop(columns=['duration'], inplace=True)

## Add new col id

In [131]:
df["id"] = df.index

Move to the left as first col

In [143]:
df = df[['id'] + [col for col in df.columns if col != 'id']]

In [144]:
df.head()

,id,type,title,director,country,release_year,rating,genre,duration_in_min,duration_in_seasons
0,0,Movie,The Grand Seduction,Don McKellar,Canada,2014,13+,"Comedy, Drama",113,0
1,1,Movie,Take Care Good Night,Girish Joshi,India,2018,13+,"Drama, International",110,0
2,2,Movie,Secrets of Deception,Josh Webber,United States,2017,13+,"Action, Drama, Suspense",74,0
3,3,Movie,Pink: Staying True,Sonia Anderson,United States,2014,13+,Documentary,69,0
4,4,Movie,Monster Maker,Giles Foster,United Kingdom,1989,13+,"Drama, Fantasy",45,0


# Load data to csv

In [147]:
df.to_csv("amazon_prime_clean.csv",index=True,encoding="utf-8")